In [138]:
import csv
import pandas as pd
import os
import subprocess
import sys
import datetime

In [298]:
data = pd.read_csv("golf-swing-dataset/data/golfDB.csv", sep=",", usecols=["id", "youtube_id", "sex", "club", "slow", "kf7"])
data = data.rename(columns={"kf7" : "impactFrame", "youtube_id" : "youtubeId"})
data.slow = data.slow.apply(lambda x: True if x == 1 else x).apply(lambda x: False if x == 0 else x)
data = data[data["slow"] == False].drop("slow", axis=1)
data.head()

,id,youtubeId,sex,club,impactFrame
0,0,f1BWA5F87Jc,f,driver,498
2,2,tA1iotgtMyc,m,driver,698
4,4,wDCKLePrwHA,f,driver,205
6,6,iPuVhnI8pJU,m,driver,332
8,8,-M5SITXMA2Y,f,driver,355


In [301]:
basePath = "/Users/metinozturk/Downloads/AI/golf-swing-dataset/videos/"
impactPreDuration = 2
impactPostDuration = 2

In [304]:
_ = [extractAudioFromVideo(x) for x in data["id"]]

In [300]:
def extractAudioFromVideo(videoIdx):
    fileName = f"{basePath}{data.loc[videoIdx].id + 1}-{data.loc[videoIdx].youtubeId}.mp4"
    frameRate = getFrameRate(fileName)
    videoDuration = getVideoDuration(fileName)
    
    if videoDuration == -1 or frameRate == -1:
        return

    impactTime = round(data.loc[videoIdx].impactFrame / frameRate, 2)

    startTime = round(0 if impactTime < impactPreDuration else impactTime - impactPreDuration, 2)
    endTime = round(videoDuration - impactTime if impactTime + impactPostDuration > videoDuration else impactTime + impactPostDuration, 2)

    startTime = datetime.timedelta(seconds=startTime)
    endTime = datetime.timedelta(seconds=endTime)


    command = (f"ffmpeg -y -i {fileName} "
           f"-ss {startTime} -to {endTime} {fileName.replace('golf-swing-dataset/videos', 'audios').replace('.mp4', '.wav')}"
          )

    subprocess.call(command, shell=True)

In [202]:
def getFrameRate(filename):
    if not os.path.exists(filename):
        return -1         
    out = subprocess.check_output(["ffprobe",filename,"-v","0","-select_streams","v","-print_format","flat","-show_entries","stream=r_frame_rate"])
    rate = out.decode().split('=')[1].strip()[1:-1].split('/')
    if len(rate)==1:
        return float(rate[0])
    if len(rate)==2:
        return float(rate[0])/float(rate[1])
    return -1

In [203]:
def getVideoDuration(filename):
    if not os.path.exists(filename):
        return -1         
    out = subprocess.check_output(["ffprobe","-v","error","-show_entries","format=duration","-of","default=noprint_wrappers=1:nokey=1", filename])
    rate = out.decode().strip()
    return float(rate)